# SQL Database Creation
This is where we will clean the data and use it to create a database of all the data to facilitate its use.

## Import Libraries

Let's start by importing the necessary libraries:

In [1]:
import pandas as pd
import sqlite3
import os
import re

## Making the Database

Now that we have imported the requisite libraries, let's create the database itself. We will call it `tourism`.

In [2]:
connection = sqlite3.connect('./data/db/tourism.db')

### Making the Tables

#### The First Table: Country

Seeing as the one common thread to all of the data is the name of the country, the first thing we should do is to make a table of the countries. We begin by reading the country data into a dataframe.

In [3]:
df_country = pd.read_csv('./data/raw-data/arrivals/Metadata_Country_API_ST.INT.ARVL_DS2_en_csv_v2_5994899.csv')

df_country.head()

,Country Code,Region,IncomeGroup,SpecialNotes,TableName,Unnamed: 5
0,ABW,Latin America & Caribbean,High income,NaN,Aruba,NaN
1,AFE,NaN,NaN,"26 countries, stretching from the Red Sea in t...",Africa Eastern and Southern,NaN
2,AFG,South Asia,Low income,The reporting period for national accounts dat...,Afghanistan,NaN
3,AFW,NaN,NaN,"22 countries, stretching from the westernmost ...",Africa Western and Central,NaN
4,AGO,Sub-Saharan Africa,Lower middle income,The World Bank systematically assesses the app...,Angola,NaN


Let's also rename these columns to something that's more python-friendly to make them easier to work with.

In [4]:
df_country.rename(columns={'Country Code': 'country_code', 
                           'Region': 'region',
                           'IncomeGroup': 'income_group',
                           'SpecialNotes': 'special_notes',
                           'TableName': 'country',
                           'Unnamed: 5': 'unnamed'}, inplace=True)

df_country.head()

,country_code,region,income_group,special_notes,country,unnamed
0,ABW,Latin America & Caribbean,High income,NaN,Aruba,NaN
1,AFE,NaN,NaN,"26 countries, stretching from the Red Sea in t...",Africa Eastern and Southern,NaN
2,AFG,South Asia,Low income,The reporting period for national accounts dat...,Afghanistan,NaN
3,AFW,NaN,NaN,"22 countries, stretching from the westernmost ...",Africa Western and Central,NaN
4,AGO,Sub-Saharan Africa,Lower middle income,The World Bank systematically assesses the app...,Angola,NaN


This next bit of code determines if the mysterious 'unnamed' field contains any data.

In [5]:
df_country[df_country.unnamed.notnull()]

,country_code,region,income_group,special_notes,country,unnamed


Since it does not contain anything, we can safely remove it. We can also remove the "special_notes" since they're not very useful for our purposes.

In [6]:
df_country.drop(columns=['special_notes', 'unnamed'], inplace=True)

df_country.head()

,country_code,region,income_group,country
0,ABW,Latin America & Caribbean,High income,Aruba
1,AFE,NaN,NaN,Africa Eastern and Southern
2,AFG,South Asia,Low income,Afghanistan
3,AFW,NaN,NaN,Africa Western and Central
4,AGO,Sub-Saharan Africa,Lower middle income,Angola


Next, let's reorder the columns to make the data easier for us to read and mentally process.

In [7]:
df_country = df_country[['country_code', 'country', 'region', 'income_group']]
df_country.head()

,country_code,country,region,income_group
0,ABW,Aruba,Latin America & Caribbean,High income
1,AFE,Africa Eastern and Southern,NaN,NaN
2,AFG,Afghanistan,South Asia,Low income
3,AFW,Africa Western and Central,NaN,NaN
4,AGO,Angola,Sub-Saharan Africa,Lower middle income


So far, it's looking much more readable; however, there are also what appear to be some aggregate regions that need to be removed for clarity. This will not be an issue, as there are also similar aggregates listed under `region` in the rows which are actually for a country.

In [8]:
df_country.drop(axis=0, index=df_country.index[df_country.region.isnull()], inplace=True)
df_country.head()

,country_code,country,region,income_group
0,ABW,Aruba,Latin America & Caribbean,High income
2,AFG,Afghanistan,South Asia,Low income
4,AGO,Angola,Sub-Saharan Africa,Lower middle income
5,ALB,Albania,Europe & Central Asia,Upper middle income
6,AND,Andorra,Europe & Central Asia,High income


Once this is turned into a table in our database, each row will be assigned an index. To make the database more efficent, we should drop the current dataframe index since it will become redundant. We can make the new dataframe index our primary key for the SQL table. 

In [9]:
df_country.set_index('country_code', drop=True, inplace=True)
df_country.head()

,country,region,income_group
country_code,,,
ABW,Aruba,Latin America & Caribbean,High income
AFG,Afghanistan,South Asia,Low income
AGO,Angola,Sub-Saharan Africa,Lower middle income
ALB,Albania,Europe & Central Asia,Upper middle income
AND,Andorra,Europe & Central Asia,High income


Now that we've cleaned this up, let's export this dataframe as a table in our `tourism` database for ease of use later.

In [10]:
df_country.to_sql('country', connection, if_exists='replace')

217

One data source down! Now let's process the others.

#### The Second Table: Arrivals

The second table we should make is probably the most apparent set of data when discussing tourism: `arrivals`. First, we create a dataframe from the relevant CSV.

In [11]:
df_arrivals = pd.read_csv('./data/raw-data/arrivals/API_ST.INT.ARVL_DS2_en_csv_v2_5994899.csv')

df_arrivals.drop(axis=1, labels=['Indicator Name', 'Indicator Code', 'Unnamed: 67'], inplace=True)
df_arrivals.rename(columns={'Country Name': 'country_name', 'Country Code': 'country_code'}, inplace=True)
df_arrivals.head()


,country_name,country_code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.667000e+06,1.739000e+06,1.832000e+06,1.758000e+06,1.863000e+06,1.897000e+06,1.951000e+06,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.442663e+07,3.573839e+07,3.531868e+07,3.764589e+07,3.825835e+07,4.118915e+07,3.982670e+07,NaN,NaN,NaN
2,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Western and Central,AFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.008582e+07,1.054462e+07,1.331168e+07,1.315078e+07,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.500000e+05,5.950000e+05,5.920000e+05,3.970000e+05,2.610000e+05,2.180000e+05,2.180000e+05,NaN,NaN,NaN


It looks like there are quite a few year columns with no entries whatsoever. Let's get rid of those.

In [12]:
df_arrivals.dropna(axis=1, how='all', inplace=True)

df_arrivals.head()

,country_name,country_code,1995,1996,1997,1998,1999,2000,2001,2002,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,9.120000e+05,9.570000e+05,9.470000e+05,9.060000e+05,9.720000e+05,1.211000e+06,1.178000e+06,1.225000e+06,...,1.469000e+06,1.481000e+06,1.667000e+06,1.739000e+06,1.832000e+06,1.758000e+06,1.863000e+06,1.897000e+06,1.951000e+06,NaN
1,Africa Eastern and Southern,AFE,1.158354e+07,1.308865e+07,1.345625e+07,1.440385e+07,1.530938e+07,1.535318e+07,1.585470e+07,1.738338e+07,...,3.165024e+07,3.274855e+07,3.442663e+07,3.573839e+07,3.531868e+07,3.764589e+07,3.825835e+07,4.118915e+07,3.982670e+07,NaN
2,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Western and Central,AFW,2.670706e+06,3.027135e+06,3.243144e+06,3.422652e+06,3.897975e+06,4.162850e+06,4.615887e+06,4.697120e+06,...,8.902380e+06,1.022103e+07,1.008582e+07,1.054462e+07,1.331168e+07,1.315078e+07,NaN,NaN,NaN,NaN
4,Angola,AGO,9.000000e+03,2.100000e+04,4.500000e+04,5.200000e+04,4.500000e+04,5.100000e+04,6.700000e+04,9.100000e+04,...,4.810000e+05,5.280000e+05,6.500000e+05,5.950000e+05,5.920000e+05,3.970000e+05,2.610000e+05,2.180000e+05,2.180000e+05,NaN


Let's also move the `country_code` column to the first row for consistency's sake.

In [13]:
arrival_columns = df_arrivals.columns.to_list()

arrival_columns[0] = 'country_code'
arrival_columns[1] = 'country_name'

df_arrivals = df_arrivals[arrival_columns]
df_arrivals.head()

,country_code,country_name,1995,1996,1997,1998,1999,2000,2001,2002,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,ABW,Aruba,9.120000e+05,9.570000e+05,9.470000e+05,9.060000e+05,9.720000e+05,1.211000e+06,1.178000e+06,1.225000e+06,...,1.469000e+06,1.481000e+06,1.667000e+06,1.739000e+06,1.832000e+06,1.758000e+06,1.863000e+06,1.897000e+06,1.951000e+06,NaN
1,AFE,Africa Eastern and Southern,1.158354e+07,1.308865e+07,1.345625e+07,1.440385e+07,1.530938e+07,1.535318e+07,1.585470e+07,1.738338e+07,...,3.165024e+07,3.274855e+07,3.442663e+07,3.573839e+07,3.531868e+07,3.764589e+07,3.825835e+07,4.118915e+07,3.982670e+07,NaN
2,AFG,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFW,Africa Western and Central,2.670706e+06,3.027135e+06,3.243144e+06,3.422652e+06,3.897975e+06,4.162850e+06,4.615887e+06,4.697120e+06,...,8.902380e+06,1.022103e+07,1.008582e+07,1.054462e+07,1.331168e+07,1.315078e+07,NaN,NaN,NaN,NaN
4,AGO,Angola,9.000000e+03,2.100000e+04,4.500000e+04,5.200000e+04,4.500000e+04,5.100000e+04,6.700000e+04,9.100000e+04,...,4.810000e+05,5.280000e+05,6.500000e+05,5.950000e+05,5.920000e+05,3.970000e+05,2.610000e+05,2.180000e+05,2.180000e+05,NaN


This also has those aggregate regions that are not needed, which we can do away with.

In [14]:
country_codes = df_country.index.to_list()

df_arrivals = df_arrivals[df_arrivals.country_code.isin(country_codes)]

df_arrivals.head()

,country_code,country_name,1995,1996,1997,1998,1999,2000,2001,2002,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,ABW,Aruba,912000.0,957000.0,947000.0,906000.0,972000.0,1211000.0,1178000.0,1225000.0,...,1469000.0,1481000.0,1667000.0,1739000.0,1832000.0,1758000.0,1863000.0,1897000.0,1951000.0,NaN
2,AFG,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,Angola,9000.0,21000.0,45000.0,52000.0,45000.0,51000.0,67000.0,91000.0,...,481000.0,528000.0,650000.0,595000.0,592000.0,397000.0,261000.0,218000.0,218000.0,NaN
5,ALB,Albania,304000.0,287000.0,119000.0,184000.0,371000.0,317000.0,354000.0,470000.0,...,2932000.0,3514000.0,3256000.0,3673000.0,4131000.0,4736000.0,5118000.0,5927000.0,6406000.0,2658000.0
6,AND,Andorra,NaN,NaN,NaN,NaN,9422000.0,10991000.0,11351000.0,11507000.0,...,7983000.0,7900000.0,7676000.0,7797000.0,7850000.0,8025000.0,8152000.0,8328000.0,8235000.0,5207000.0


The dataframe is already looking much better, but it's really awkward to work with the data in its current state.

The data is currently in a **wide-format**, with a year for each column. To convert the dataframe into a **long format**, we can use the `melt` function of pandas. This will essentially "unpivot" the years and the number of arrivals for each year back into the table. Unfortunately for us, it's less human-readable this way, but the data is much easier to manipulate with SQL.

In [15]:
df_arrivals = df_arrivals.melt(id_vars=['country_code', 'country_name'], var_name='year', value_name='arrivals')

df_arrivals.head()

,country_code,country_name,year,arrivals
0,ABW,Aruba,1995,912000.0
1,AFG,Afghanistan,1995,NaN
2,AGO,Angola,1995,9000.0
3,ALB,Albania,1995,304000.0
4,AND,Andorra,1995,NaN


One small issue that remains is that the years are of the type `string` since they were the titles of the columns. We can quickly recast them as integers.

In [16]:
df_arrivals = df_arrivals.astype({'year': 'int'})

df_arrivals.year.head()

0    1995
1    1995
2    1995
3    1995
4    1995
Name: year, dtype: int64

Much improved! Let's just make a few final tweaks to the dataframe, starting with dropping the country name. 

To increase efficiency, we will be using `country_code` as the foreign key, thus the `country_name` column will be redundant. We should also set `country_code` as the index to avoid duplicating the index in the SQL table, as we did with the `country` table. 

Finally, we should sort by country code, then by year for consistency and legibility.

In [17]:
df_arrivals.drop(columns=['country_name'], inplace=True)
df_arrivals.set_index('country_code', drop=True, inplace=True)
df_arrivals.sort_values(by=['country_code', 'year'], inplace=True)
df_arrivals.head()

,year,arrivals
country_code,,
ABW,1995,912000.0
ABW,1996,957000.0
ABW,1997,947000.0
ABW,1998,906000.0
ABW,1999,972000.0


Now let's get this one saved as a database as well.

In [18]:
df_arrivals.to_sql('arrivals', connection, if_exists='replace')

5642

#### The Remaining Tables
Now that we have created one of the data tables, we have a better idea of what needs to be done to the other sets of data. Let's make a dataframe for each one that we have left.

In [19]:
df_departures = pd.read_csv('./data/raw-data/departures/API_ST.INT.DPRT_DS2_en_csv_v2_5996775.csv')
df_expenditures = pd.read_csv('./data/raw-data/expenditures/API_ST.INT.XPND.CD_DS2_en_csv_v2_5996767.csv')
df_gdp = pd.read_csv('./data/raw-data/gdp/API_NY.GDP.MKTP.KD_DS2_en_csv_v2_5994841.csv')
df_gdp_per_capita = pd.read_csv('./data/raw-data/gdp-per-capita/API_NY.GDP.PCAP.KD_DS2_en_csv_v2_5994684.csv')
df_income = pd.read_csv('./data/raw-data/income/API_NY.ADJ.NNTY.PC.CD_DS2_en_csv_v2_5996043.csv')
df_receipts = pd.read_csv('./data/raw-data/receipts/API_ST.INT.RCPT.CD_DS2_en_csv_v2_5996774.csv')
df_population = pd.read_csv('./data/raw-data/population/API_SP.POP.TOTL_DS2_en_csv_v2_6011311.csv')

Because we could see in the exploration of the data that all of the CSV files have the same format, we can process these in bulk the same way that we did for the `arrivals` table that we just created. Let's start by adding the dataframes we just made to a list, and then create another list with the titles we want for our SQL tables.

In [20]:
dataframes = [df_departures, df_expenditures, df_gdp, df_gdp_per_capita, df_income, df_receipts, df_population]
table_titles = ['departures', 'expenditures', 'gdp', 'gdp_per_capita', 'income', 'receipts', 'population']

Now let's process the remaining dataframes into tables!

In [21]:
for title, df in zip(table_titles, dataframes):
    df.drop(columns=['Country Name', 'Indicator Name', 'Indicator Code'], inplace=True)
    df.dropna(axis=1, how='all', inplace=True)
    df.rename(columns={'Country Code': 'country_code'}, inplace=True)
    df = df[df.country_code.isin(country_codes)]
    df = df.melt(id_vars=['country_code'], var_name='year', value_name=title)
    df = df.astype({'year': 'int'})
    df.set_index('country_code', drop=True, inplace=True)
    df.sort_values(by=['country_code', 'year'], inplace=True)
    df.to_sql(title, connection, if_exists='replace')

### Tidying Up
Unfortunely, there is a minor limitation when creating the tables in the database the way that we did: there was no option to set either a primary or foreign key. This will likely not be a major issue with our data, but it could be for larger or more complex sets. Let's correct that problem! We begin by connecting to the database.

In [22]:
connection = sqlite3.connect('./data/db/tourism.db')
cursor = connection.cursor()

Now, using the existing tables, let's regenerate new ones with keys, starting with the `country` table.

In [23]:
title = 'country'
old_table = 'country_old'
rename_table = f'''
    ALTER TABLE {title}
    RENAME TO {old_table};
    '''
create_table = f'''
    CREATE TABLE {title} (
        country_code VARCHAR(4) PRIMARY KEY,
        country VARCHAR(128),
        region VARCHAR(256),
        income_group VARCHAR(128)
    );
    '''
insert_data = f'''
    INSERT INTO {title} (country_code, country, region, income_group)
    SELECT country_code, country, region, income_group
    FROM {old_table};
    '''
drop_table = f'''
    DROP TABLE IF EXISTS {old_table};
    '''
cursor.execute(rename_table)
cursor.execute(create_table)
cursor.execute(insert_data)
cursor.execute(drop_table)
connection.commit()

Now let's make a list of the tables we still have to change, and then use it to change them all at once.

In [24]:
tables = ['arrivals', 'departures', 'expenditures', 'gdp', 'gdp_per_capita', 'income', 'receipts', 'population']

for title in tables:
    column = title
    old_table = f'{title}_old'
    rename_table = f'''
    ALTER TABLE {title}
    RENAME TO {old_table};
    '''
    create_table = f'''
    CREATE TABLE {title} (
        country_code VARCHAR(4),
        year INT,
        {column} REAL,
        FOREIGN KEY (country_code) REFERENCES country(country_code)
    );
    '''
    insert_data = f'''
    INSERT INTO {title} (country_code, year, {column})
    SELECT country_code, year, {column}
    FROM {old_table};
    '''
    drop_table = f'''
    DROP TABLE IF EXISTS {old_table};
    '''
    cursor.execute(rename_table)
    cursor.execute(create_table)
    cursor.execute(insert_data)
    cursor.execute(drop_table)

connection.commit()
connection.close()

There we have it! Our database has been successfully created!